In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

FILENAME = "cleaned_content_based.csv"
DATA_PATH = os.getenv("FILES_LOCATION")
RECOMMENDER_TYPE = "content_based"
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, DATA_PATH, "PNG", RECOMMENDER_TYPE)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, extension="png", resolution=300):  # Función para guardar las figuras que se vayan generando
    img_path = os.path.join(IMAGES_PATH, fig_id + "." + extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(img_path, format=extension, dpi=resolution)

In [3]:
from matplotlib import pyplot as plt

# Configuración de parámetros de matplotlib

plt.rc("font", size=14)
plt.rc("axes", labelsize=14, titlesize=14)
plt.rc("legend", fontsize=14)
plt.rc("xtick", labelsize=10)
plt.rc("ytick", labelsize=10)

In [4]:
df = pd.read_csv(os.path.join(DATA_PATH, "CSV", FILENAME), low_memory=False)

# Content Based Filtering

El primer modelo que hemos creado ha sido un recomendador muy simple, basado en las películas y su popularidad, con un filtro del género de la película. Sin embargo, este sistema recomienda las mismas películas a todos los usuarios, por lo que si un usuario quiere recomendaciones sobre películas de Romance, siempre recibirá las mismas recomendaciones que otros usuarios que busquen ese mismo género.

En caso de que una persona quisiera conocer películas de un mismo director, actor o que tenga una trama similar a otra pero con diferentes géneros, el recomendador simple que hemos elaborado no serviría. Vamos a hacer dos tipos de recomendadores:

- Basado en el contenido de la descripción de la película
- Basado en los metadatos (director, actores, géneros...)

De esta forma desarrollaremos dos sistemas de recomendación superiores al que hicimos anteriormente.

## Basado en la descripción

Durante el procesamiento de los datos en _notebook_ EDA00, hicimos una característica que englobaba la reseña y el _tagline_ de las películas (en caso de haber _tagline_). A partir de esta descripción, podemos realizar un sistema de recomendación basado en la similaridad de las películas. Recordemos que en nuestro caso la característica que engloba todo se llama _**description**_.

In [5]:
df.head(3)

,genres,id,title,overview,description,popularity,vote_average,vote_count,cast,keywords,director,metadata
0,"['animation', 'comedy', 'family']",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...",21.946943,7.7,5415.0,"['tomhanks', 'timallen', 'donrickles']","['jealousi', 'toy', 'boy', 'friendship', 'frie...",['johnlasseter'],jealousi toy boy friendship friend rivalri boy...
1,"['adventure', 'fantasy', 'family']",8844,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...,17.015539,6.9,2413.0,"['robinwilliams', 'jonathanhyde', 'kirstendunst']","['boardgam', 'disappear', ""basedonchildren'sbo...",['joejohnston'],boardgam disappear basedonchildren'sbook newho...
2,"['romance', 'comedy']",15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...,11.712900,6.5,92.0,"['waltermatthau', 'jacklemmon', 'ann-margret']","['fish', 'bestfriend', 'duringcreditssting']",['howarddeutch'],fish bestfriend duringcreditssting waltermatth...


In [6]:
# df = df.dropna(subset="metadata")

Lo primero que vamos a realizar es una tokenización del texto, eliminando los _stopwords_ y después le vamos a pasar un _stemmer_ tras tokenizar el texto.

In [7]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
import re

stop_w = set(stopwords.words("english"))  # Stopwords
remove_non_alpha = lambda x: re.sub(r"[^a-zA-Z]", " ", x)  # Elimina aquello que no sean palabras
tokenize = lambda x: nltk.tokenize.word_tokenize(x)  # Tokeniza cada entrada
stemmer = SnowballStemmer("english")
stemmer_fn = lambda desc: " ".join([stemmer.stem(w) for w in desc if w.lower() not in stop_w])  # Paso de estandarización

In [8]:
for fn in (remove_non_alpha, tokenize, stemmer_fn):
    df["description"] = df["description"].transform(func=fn)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english", min_df=0.)  # Redundancia de stopwords
tfidf_matrix = tfidf.fit_transform(df["description"])

In [10]:
tfidf_matrix.shape

(41361, 921488)

Como hemos utilizado _TfidfVectorizer_, vamos a computar la similaridad con el kernel lineal, ya que el producto interno nos dará como resultado la similaridad coseno. Utilizando el kernel lineal la computación será más rápida.

In [11]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:
indices_mapper = pd.Series(df.index, index=df["title"]).drop_duplicates()

In [13]:
movie = "Avengers: Age of Ultron"
idx = indices_mapper[movie]
print(idx)
scores = list(enumerate(cosine_sim[idx]))
scores = sorted(scores, key=lambda x: x[1], reverse=True)
scores = scores[0:11]
idx_ = [s[0] for s in scores]
df["title"].iloc[idx_]

24813


24813                              Avengers: Age of Ultron
25223                          Tarzan's Greatest Adventure
12143                                             Iron Man
26054                                           Calculator
14526                                           Iron Man 2
28319                                    A Deadly Adoption
19711                                           Iron Man 3
30240                                                Hawks
36118    Fittest On Earth (The Story Of The 2015 Reebok...
17815                                                Verbo
17113                  Behind Enemy Lines II: Axis of Evil
Name: title, dtype: object

Vemos que el recomendador funciona como se esperaba. Vamos a guardar los parámetros y a crear las funciones necesarias para el sistema.

In [14]:
import pickle

MODELS_PATH = os.getenv("MODELS_PATH")
os.makedirs(os.path.join(MODELS_PATH, RECOMMENDER_TYPE), exist_ok=True)

with open(os.path.join(MODELS_PATH, RECOMMENDER_TYPE, "tfidf_description_matrix.pickle"), "wb") as f:
    pickle.dump(cosine_sim, f, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
import ast
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pickle
import numpy as np


load_dotenv()
DEFAULT_PATH =os.path.join(
    os.getenv("MODELS_PATH"),
    RECOMMENDER_TYPE,
    "tfidf_description_matrix.pickle"
)


def load_param_matrix(path=DEFAULT_PATH):
    """Carga el modelo basado en descripción de tfidf
    Args:
        path (str): ruta en la que se encuentra el fichero con los parámetros
    Returns:
        param_matrix (numpy ndarray): matriz de numpy que contiene la similaridad entre cada película
    """
    
    with open(path, "rb") as f:
        param_matrix = pickle.load(f)
    return param_matrix


def preprocess_data(dataframe, col="description"):
    """Pre-procesamiento de los datos previo al entrenamiento.
    Args:
        dataframe (pandas DataFrame): DataFrame con la columna a procesar
        col (str): nombre de la columna sobre la que realizar el procesado. Default description
    Returns:
        dataframe (pandas DataFrame): datos procesados con nltk
    """
    
    stop_w = set(stopwords.words("english"))  # Selecciona los stopwords
    
    remove_non_alpha_fn = lambda x: re.sub(r"[^a-zA-Z]", " ", x)  # Elimina aquello que no sean palabras
    tokenize_fn = lambda x: nltk.tokenize.word_tokenize(x)  # Tokeniza cada entrada
    
    stemmer = SnowballStemmer("english")
    stemmer_fn = lambda desc: " ".join([stemmer.stem(w) for w in desc if w.lower() not in stop_w])  # Paso de estandarización

    # Aplica cada función a la columna
    for fn in (remove_non_alpha_fn, tokenize_fn, stemmer_fn):
        dataframe[col] = dataframe[col].transform(func=fn)
    return dataframe


def train_content_recommender(dataframe, preprocess_fn, col="description", count_vec=False):
    """Entrenamiento del recomendador
    Args:
        dataframe (pandas DataFrame): DataFrame con los datos para entrenar
        preprocess_fn (function): función que realiza el procesado de los datos previo al entrenamiento
        col (str): nombre de la columna sobre la que realizar el entrenamiento. Default description
        count_vec (bool): booleano que indica si se usa o no CountVectorizer. Default False
    Returns:
        param_matrix (numpy ndarray): matriz de numpy que contiene la similaridad entre cada película
    """

    if count_vec:
        dataframe[col] = dataframe[col].fillna("")
        count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0., stop_words="english")
        count_vec_matrix = count.fit_transform(processed_data[col])
        return cosine_similarity(count_vec_matrix, count_vec_matrix)
    processed_data = preprocess_fn(dataframe, col)
    tfidf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english", min_df=0.)  # Redundancia de stopwords
    tfidf_matrix = tfidf.fit_transform(processed_data[col])

    return linear_kernel(tfidf_matrix, tfidf_matrix)


def get_description_recommendation(movie, top_n= 10, dataframe=df, preprocess_fn=preprocess_data, 
                                   col="description", count_vec=False, path=DEFAULT_PATH):
    """Recomendador basado en la descripción de películas
    Args:
        movie (str): títutlo de la película sobre la que realizará las recomendaciones
        top_n (int): número de películas que recomienda al usuario. Default 10
        dataframe (pandas DataFrame): DataFrame con los datos para entrenar y los títulos a recomendar
        preprocess_fn (function): función que realiza el procesado de los datos previo al entrenamiento
        col (str): nombre de la columna sobre la que realizar el entrenamiento en caso de ser necesario. Default description
        count_vec (bool): booleano que indica si se usa o no CountVectorizer en el entrenamiento. Default False
        path (str): ruta en la que se encuentra el fichero con los parámetros
    Returns:
        movies list (list): lista con los títulos de las top_n películas más similares
    """
    if not isinstance(movie, str):  # Aseguramos que el usuario introduzca una string y no indicamos que no es tipo str
        raise ValueError("Película no encontrada.")

    # Intentamos cargar los parámetros. En caso de no haberlos, realizamos el entrenamiento
    try:
        param_matrix = load_param_matrix(path)
    except:
        param_matrix = train_content_recommender(df, preprocess_description, col, count_vec)

    mapper = pd.Series(dataframe.index, index=dataframe["title"]).drop_duplicates()  # Para mapear ids con títulos
    scores = np.argsort(param_matrix[mapper[movie]])[::-1]  # Obtiene los índices de las películas con mayor similaridad
    movie_indices = scores[1:top_n + 1] # Top_n películas más similares, eliminando a movie

    return list(dataframe["title"].iloc[movie_indices].values)

In [16]:
get_description_recommendation("Avengers: Age of Ultron")

["Tarzan's Greatest Adventure",
 'Iron Man',
 'Calculator',
 'Iron Man 2',
 'A Deadly Adoption',
 'Iron Man 3',
 'Hawks',
 'Fittest On Earth (The Story Of The 2015 Reebok CrossFit Games)',
 'Verbo',
 'Behind Enemy Lines II: Axis of Evil']

## Basado en metadatos

Hemos visto cómo funciona el recomendador basado en la descripción, utilizando el método de Tfidf. Ahora vamos a utilizar la columna que hicimos en el EDA01 para crear un recomendador basado en los metadatos como

In [22]:
df["metadata"] = df["metadata"].fillna("")

In [23]:
count_vec = CountVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0., stop_words="english")
count_vec_matrix = count_vec.fit_transform(df["metadata"])
param_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)

with open(os.path.join(MODELS_PATH, RECOMMENDER_TYPE, "count_metadata_matrix.pickle"), "wb") as f:
    pickle.dump(param_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
indices_mapper = pd.Series(df.index, df["title"]).drop_duplicates()

In [26]:
movie = "Avengers: Age of Ultron"
idx = indices_mapper[movie]
print(idx)
scores = list(enumerate(param_matrix[idx]))
scores = sorted(scores, key=lambda x: x[1], reverse=True)
scores = scores[0:11]
idx_ = [s[0] for s in scores]
df["title"].iloc[idx_]

24813


24813                Avengers: Age of Ultron
24821             Captain America: Civil War
24817                                Ant-Man
24819                         Thor: Ragnarok
20735                   Thor: The Dark World
21714    Captain America: The Winter Soldier
19711                             Iron Man 3
16633     Captain America: The First Avenger
24820         Guardians of the Galaxy Vol. 2
14526                             Iron Man 2
2486                             Superman II
Name: title, dtype: object

In [28]:
get_description_recommendation("Avengers: Age of Ultron", col="metadatos", count_vec=True, 
                               path=os.path.join(os.getenv("MODELS_PATH"), RECOMMENDER_TYPE, "count_metadata_matrix.pickle"))

['Captain America: Civil War',
 'Ant-Man',
 'Thor: Ragnarok',
 'Thor: The Dark World',
 'Captain America: The Winter Soldier',
 'Iron Man 3',
 'Captain America: The First Avenger',
 'Guardians of the Galaxy Vol. 2',
 'Iron Man 2',
 'Superman II']